# Guided Studies into Financial Management
## Index Revisions and Stock Returns

### Colaborators
Dennis Blaufuss,
Lars Wrede,
Nicolas Kepper,
Sophie Merl,
Philipp Voit

### Instructor
Dr. Stefan Scharnoski

### Summary 
HIER MÜSSEN WIR NOCH EINE ZUSAMMENFASSUNG DER ERGEBNISSE SCHREIBEN - WIE ABSTRACT

---

## Table of Content:
1. [Data Proprocessing](#0-bullet)
* [Import Data](#first-bullet)
* [Calculate Daily Returns](#2-bullet)
* [Data Quality Checks](#3-bullet)
* [Descriptive Statistics](#4-bullet)
2. [Price Pressure](#5-bullet)
3. [Investor Attention](#6-bullet)
4. [Systematic Risk and Liquidity](#7-bullet)
* [Systematic Risk - Included Stocks](#8-bullet)
* [Systematic Risk - Excluded Stocks](#9-bullet)
* [Systematic Risk - DAX 30 to DAX 40](#10-bullet)
___

In [1]:
import pandas as pd
import numpy as np 
import datetime

import yfinance as yf

import performanceanalytics.table.table as pat
from performanceanalytics.charts import performance_summary
import statistics
import statsmodels.formula.api as smf

# 1. Data Preprocessing <a class="anchor" id="0-bullet"></a>

## Import Data <a class="anchor" id="first-bullet"></a>

In [2]:
''' Import all relevant data.
Parameters
----------
:DAX_aufgenommen: df
    All stocks included in the DAX from 2010 until 2021.
:DAX_ausgeschieden: df
    All stocks excluded from the DAX from 2010 until 2021.
:stock_data: df
    Contains stock ticker as well as names of the stocks of all DAX stock from 22-03-01.
:benchmark:  df
    The MSCI Germany Index was used as a proxy for the market portfolio. 
    This index contains a large number of M-DAX and DAX stocks and is therefore more 
    broadly structured than the DAX. However, the high weight of the DAX shares in the index is problematic, 
    so that it is to be expected that the actual influence is underestimated.
:index_compositions: df
    Contains the deletions/ additions as well as date of change/ announcements & Merger/Spin-Off Information.
-------
'''
DAX_included = pd.read_csv('DAX_included_2010-2021.csv', sep = ';')
DAX_excluded = pd.read_csv('DAX_excluded_2010-2021.csv', sep = ';')
stock_data = pd.read_csv('Companies_Ticker.csv', sep = ';')
benchmark = pd.read_csv('DAX_Kurs.csv', sep = ';')
announcements = pd.read_csv('Historical_Index_Compositions.csv', sep = ';')
stocks_data = pd.read_csv('stocks-historical-data.csv', sep = ';')
#index_compositions = pd.read_csv('Historical_Index_Compositions.csv', sep = ';')

## Creating Info DataFrame <a class="anchor" id="2-bullet"></a>

* Joining all Stocks that where at some point in time after 2010 in the DAX.
* Stocks that were in the Dax the whole time have will have all movement related columns set to N/A.
* Both dates are formatted in pandas datetime format.

In [3]:
exclusions = DAX_excluded.rename(columns={'Ausgeschieden': 'Date'})
exclusions['Type'] = 'Excluded'
exclusions['Date'] = pd.to_datetime(exclusions['Date'])

inclusions = DAX_included.rename(columns={'Aufgenommen': 'Date'})
inclusions['Type'] = 'Included'
inclusions['Date'] = pd.to_datetime(inclusions['Date'])

rest = stock_data[stock_data['Symbol'].isin(inclusions['Symbol']) == False]

info_df = pd.concat([exclusions, inclusions, rest])
info_df.reset_index(drop=True, inplace=True)

announcements.rename(columns={'Date of change': 'Date', 'Date of announcement': 'Announcement'}, inplace=True)
announcements['Date'] = pd.to_datetime(announcements['Date'])

info_df = info_df.join(announcements.set_index('Date')['Announcement'], on='Date')
info_df['Announcement'] = pd.to_datetime(info_df['Announcement'])
info_df

,Date,Symbol,Firmenname,Type,Announcement
0,2010-06-21,SZG.DE,Salzgitter AG,Excluded,2010-04-06
1,2015-09-21,LXS.DE,LANXESS Aktiengesellschaft,Excluded,2015-03-09
2,2016-03-21,SDF.DE,K+S Aktiengesellschaft,Excluded,2016-03-03
3,2018-03-19,PSM.DE,ProSiebenSat.1 Media SE,Excluded,2018-05-03
4,2018-09-24,CBK.DE,Commerzbank AG,Excluded,2018-05-09
5,2019-09-23,TKA.DE,thyssenkrupp AG,Excluded,2019-04-09
6,2020-06-22,LHA.DE,Deutsche Lufthansa AG,Excluded,2020-04-06
7,2020-08-24,WDI.HM,Wirecard AG,Excluded,2020-08-19
8,2021-03-22,BEI.DE,Beiersdorf Aktiengesellschaft,Excluded,2021-03-03
9,2021-10-29,DWNI.DE,Deutsche Wohnen SE,Excluded,2021-10-26


## Creating Stockdata DataFrame <a class="anchor" id="2-bullet"></a>

* Placeholder

In [ ]:
stocks_data

## Creating Stockdata DataFrame (Yahoo finance backup) <a class="anchor" id="2-bullet"></a>

In [4]:
'''Get data from yahoo finance.

Parameters
----------
:data_joined:  df
    Contains Adj CLose for all stocks.
-------
'''
tickers = pd.read_csv('Companies_Ticker.csv', sep = ';')

start_date = '2010-01-01'
end_date = '2022-03-01'
i = 0

for key in tickers['Symbol']:
    data = yf.download(tickers['Symbol'][i], start=start_date, end=end_date, progress=False)
    data.reset_index(inplace=True)
    data.drop(['High', 'Low', 'Open', 'Close', 'Volume'], axis = 1, inplace=True)
    data.rename(columns={'Adj Close': tickers['Symbol'][i]}, inplace=True)
    
    if i > 0:
        data_joined = data_joined.join(data.set_index('Date'), on='Date', how='outer')
    else:
        data_joined = data
    i += 1

data_joined.sort_values(by='Date', inplace=True)   
data_joined

,Date,^GDAXI,ADS.DE,AIR.DE,ALV.DE,BAS.DE,BAYN.DE,BMW.DE,BNR.DE,BEI.DE,...,RWE.DE,SAP.DE,SRT3.DE,SIE.DE,ENR.DE,SHL.DE,SY1.DE,VOW3.DE,VNA.DE,ZAL.DE
0.0,2010-01-04,6048.299805,31.746376,11.494962,50.552078,25.849697,38.053692,20.743076,NaN,39.848839,...,40.021980,27.285362,3.707397,41.060326,NaN,NaN,12.520763,49.368145,NaN,NaN
1.0,2010-01-05,6031.859863,32.748112,11.412854,50.706219,25.457769,37.358742,20.911348,NaN,39.638641,...,39.766632,27.041008,3.585296,41.085407,NaN,NaN,12.524799,48.411404,NaN,NaN
2.0,2010-01-06,6034.330078,32.484276,11.490857,51.100182,25.619158,37.122589,21.234957,NaN,39.608612,...,39.691181,27.521441,3.618596,41.235935,NaN,NaN,12.928304,49.750843,NaN,NaN
3.0,2010-01-07,6019.359863,32.768715,11.589385,50.512100,25.449129,36.636799,21.422646,NaN,39.209644,...,39.522873,28.250376,3.516477,41.725117,NaN,NaN,12.839533,50.324905,NaN,NaN
4.0,2010-01-08,6037.609863,32.649170,11.778231,50.238041,25.371321,36.312939,21.134636,NaN,38.077118,...,39.464832,28.449173,3.529796,41.988529,NaN,NaN,12.960585,50.868324,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076.0,2022-02-22,14693.000000,223.050003,114.019997,206.600006,65.269997,52.410000,90.930000,76.419998,86.720001,...,37.790001,100.059998,388.600006,132.039993,18.126492,53.080002,102.300003,188.699997,45.990002,57.939999
3077.0,2022-02-23,14631.360352,217.899994,113.779999,206.350006,64.860001,52.950001,91.620003,76.000000,88.459999,...,37.540001,99.599998,386.899994,131.539993,18.051878,53.779999,102.699997,187.080002,45.709999,56.000000
3078.0,2022-02-24,14052.099609,210.149994,108.900002,200.199997,61.060001,50.599998,84.989998,71.559998,86.559998,...,37.709999,97.480003,383.000000,124.180000,19.385000,53.900002,102.900002,176.100006,44.410000,55.500000
3079.0,2022-02-25,14567.230469,213.850006,115.980003,207.449997,60.430000,52.880001,88.019997,75.220001,89.660004,...,40.000000,101.400002,388.299988,129.720001,19.434999,55.320000,105.550003,185.320007,46.779999,55.959999


## To be deleted, wenn alle analysne angepasst sind <a class="anchor" id="2-bullet"></a>

In [5]:
''' Pulls time series data for stocks on a daily basis from 2010-01-01 until 2022-03-01.

Parameters
----------
:stock_dict:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_dict = {}
for s in stock_data['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_dict[s] = yf.download(s, start='2010-01-01', end='2022-03-01', progress=False)
# Concatenate all data
stocks_as_df = pd.concat(stock_dict, axis = 0)

In [6]:
''' Pulls time series data for stocks on a daily basis from 2009-06-21 until 2022-03-01.

Parameters
----------
:stock_included:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_included = {}
for s in DAX_included['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_included[s] = yf.download(s, start='2009-06-21', end='2022-03-01', progress=False)
stock_included_as_df = pd.concat(stock_included, axis = 0)

In [7]:
''' Pulls time series data for stocks on a daily basis from 2009-06-21 until 2022-03-01.

Parameters
----------
:stock_included:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_excluded = {}
for s in DAX_excluded['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_excluded[s] = yf.download(s, start='2009-06-21', end='2022-03-01', progress=False)
stock_excluded_as_df = pd.concat(stock_excluded, axis = 0)

## Calculate Daily Returns(to be deleted) <a class="anchor" id="2-bullet"></a>

In [8]:
''' Transform daily price data to daily returns.
Parameters
----------
:returns_daily:  dict
    Contains the stock symbols as key and the daily returns as values.
:returns_daily_excluded: dict
    Contains the stock symbols as key and the daily returns as values.
:returns_daily_included: dict
    Contains the stock symbols as key and the daily returns as values.
:benchmark: df
    Contains daily returns from the benchmark.
-------
'''
returns_daily = {}
for s in stock_data['Symbol']:
    returns_daily[s] = stock_dict[s]['Adj Close'].pct_change()
returns_daily_included = {}
for s in DAX_included['Symbol']:
    returns_daily_included[s] = stock_included[s]['Adj Close'].pct_change()
DAX_included['Aufgenommen'] = pd.to_datetime(DAX_included['Aufgenommen'], format='%d.%m.%Y')

returns_daily_excluded = {}
for s in DAX_excluded['Symbol']:
    returns_daily_excluded[s] = stock_excluded[s]['Adj Close'].pct_change()
DAX_excluded['Ausgeschieden'] = pd.to_datetime(DAX_excluded['Ausgeschieden'], format='%d.%m.%Y')
    
benchmark['Umtauschdatum'] = pd.to_datetime(benchmark['Umtauschdatum'], format='%d.%m.%y')
benchmark = pd.DataFrame(benchmark['Schlusskurs'].astype(float).pct_change()).set_index(benchmark['Umtauschdatum'])

## Data Quality Checks (needs to be updated) <a class="anchor" id="3-bullet"></a>

In [10]:
'''Check if stocks_as_df contains NA or zeros in Volume & Adjusted Close.

Parameters
----------
:stocks_as_df:  df
    Contains the time series data as one df.
:stocks_as_df_Volume_is_0:  df
    Contains the rows where Volume == 0.
-------
'''

stocks_as_df_has_nan = np.isnan(np.sum(stocks_as_df)) #no NAs

#(stocks_as_df < 0).any()
#(stocks_as_df = 0).any()

stocks_as_df_Volume_is_0 = stocks_as_df.loc[stocks_as_df["Volume"] == 0] #2774 times 0
stocks_as_df_AdjClose_is_0 = stocks_as_df.loc[stocks_as_df["Adj Close"] == 0] #never 0

#stock_dict['ZAL.DE'] #spot the 0 for '2014-10-06'

In [12]:
'''Interpolate Volume using 'spline'.

Parameters
----------
:stock_dict_replaced_all:  dict
    Contains the time series data as stock_dict but with interpolated values for 'Volume'.

:stock_dict_replaced:  dict
    Contains the time series data as stock_dict but with interpolated values for 'Volume' only when 'Close' is not equal to 'Open'.
-------
'''
#stock_dict_replaced_all = stock_dict
#for key in stock_dict_replaced_all:
#    stock_dict_replaced_all[key].loc[stock_dict_replaced_all[key]['Volume'] == 0, 'Volume'] = np.nan
#    stock_dict_replaced_all[key]['Volume'].interpolate(method ='spline', order = 2, inplace=True)

#stock_dict_replaced_all['^GDAXI'].loc[:'2018-04-27'] #<- should now have an interpolated value at '2018-04-27'
#stock_dict_replaced_all['ZAL.DE'].loc[:'2017-06-05'] #<- should now have interpolated value at '2017-06-05'

stock_dict_replaced = stock_dict
for key in stock_dict_replaced:
    stock_dict_replaced[key].loc[(stock_dict_replaced[key]['Volume'] == 0) & (stock_dict_replaced[key]['Open'] != stock_dict_replaced[key]['Close']), 'Volume'] = np.nan
    stock_dict_replaced[key]['Volume'].interpolate(method ='spline', order = 2, inplace=True)

#stock_dict_replaced['^GDAXI'].loc[:'2018-04-27'] #<- should now have an interpolated value at '2018-04-27'
#stock_dict_replaced['ZAL.DE'].loc[:'2017-06-05'] #<- should now NOT have interpolated value at '2017-06-05'


In [13]:
'''Check date index (to be deleted when stock_dict is out of use)

Parameters
----------
:dates:  df
    Contains the 'Date' index from each stock. 
-------
'''

stock_dict_check_idx = stock_dict

dates = pd.DataFrame()

for key in stock_dict_check_idx:
    stock_dict_check_idx[key].reset_index(inplace=True)
    dates[key] = stock_dict_check_idx[key]['Date']

#dates.to_excel("dates.xlsx")

In [14]:
'''Check if Adj Close in stocks_as_df differs from previous/ following day.

Parameters
----------
:stocks_as_df:  df
    Contains the time series data as one df.
:stocks_as_df_adjclose_peak_bottom:  dataframe
    Contains the rows where Adj. Close differs
-------
'''
stocks_as_df_adjclose_peak_bottom_list = []
n = 1

while n < len(stocks_as_df)-1:
    if abs(stocks_as_df['Adj Close'][n] -
           statistics.mean([stocks_as_df['Adj Close'][n-1],
                            stocks_as_df['Adj Close'][n+1]])) > .5 * stocks_as_df['Adj Close'][n]:
        stocks_as_df_adjclose_peak_bottom_list.append(stocks_as_df.iloc[n])

    n += 1

stocks_as_df_adjclose_peak_bottom = pd.DataFrame(stocks_as_df_adjclose_peak_bottom_list) #29 times 

## Descriptive Statistics of the whole Dataset (needs to be updated) <a class="anchor" id="4-bullet"></a>

In [16]:
''' Calculating measures of location, statistical dispersion and shape.
Parameters
----------
:des_stat:  dataframe
    Contains the descriptive statistics.
-------
'''

des_stat = pd.DataFrame(columns=stock_data['Symbol'], 
                        index=['Observations', 'NAs', 'Minimum', 'Quartile 1', 'Median', 
                               'Artithmetic Mean', 'Geometric Mean', 'Quartile 3', 'Maximum', 'SE Mean',
                               'LCL Mean (.95)', 'UCL Mean (.95)', 'Variance', 'Stdev', 'Skewness','Kurtosis'])

for s in stock_data['Symbol']:
    df = pd.DataFrame(returns_daily[s])
    des_stat[s] = pat.stats_table(df, manager_col=0)
des_stat

Symbol,^GDAXI,ADS.DE,AIR.DE,ALV.DE,BAS.DE,BAYN.DE,BMW.DE,BNR.DE,BEI.DE,CON.DE,...,RWE.DE,SAP.DE,SRT3.DE,SIE.DE,ENR.DE,SHL.DE,SY1.DE,VOW3.DE,VNA.DE,ZAL.DE
Observations,3080.000000,3085.000000,3085.000000,3085.000000,3085.000000,3085.000000,3085.000000,3025.000000,3086.000000,3085.000000,...,3085.000000,3085.000000,3085.000000,3085.000000,360.000000,982.000000,3085.000000,3085.000000,2187.000000,1877.000000
NAs,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Minimum,-0.122386,-0.153704,-0.215624,-0.153277,-0.117939,-0.140271,-0.129714,-0.091662,-0.098130,-0.174502,...,-0.172973,-0.219375,-0.122452,-0.126960,-0.166303,-0.089523,-0.091072,-0.198184,-0.084064,-0.131981
Quartile 1,-0.005333,-0.008441,-0.009765,-0.006613,-0.008043,-0.008795,-0.008869,-0.007579,-0.006143,-0.010885,...,-0.009771,-0.006683,-0.009468,-0.007941,-0.013067,-0.007322,-0.007104,-0.010770,-0.006763,-0.010127
Median,0.000799,0.000105,0.000553,0.000539,0.000567,0.000000,0.000216,0.000551,0.000272,0.000492,...,0.000278,0.000818,0.001197,0.000426,-0.000632,0.000491,0.000597,0.000140,0.000878,0.000875
Artithmetic Mean,0.000364,0.000769,0.000993,0.000579,0.000402,0.000252,0.000632,0.000671,0.000340,0.000564,...,0.000221,0.000534,0.001757,0.000486,0.000254,0.000787,0.000801,0.000658,0.000760,0.000847
Geometric Mean,0.000283,0.000616,0.000747,0.000451,0.000269,0.000098,0.000463,0.000549,0.000266,0.000327,...,0.000012,0.000425,0.001514,0.000365,-0.000059,0.000655,0.000694,0.000416,0.000656,0.000543
Quartile 3,0.006680,0.009800,0.011406,0.007958,0.009276,0.009589,0.009753,0.008758,0.006822,0.011975,...,0.010358,0.008453,0.012813,0.008573,0.014430,0.009778,0.008665,0.011554,0.008083,0.013182
Maximum,0.109759,0.111834,0.206724,0.158039,0.107292,0.087368,0.144724,0.086927,0.076040,0.135634,...,0.165748,0.125491,0.181979,0.115594,0.102650,0.096943,0.103354,0.190461,0.112109,0.237586
SE Mean,0.000229,0.000315,0.000399,0.000288,0.000294,0.000315,0.000331,0.000284,0.000220,0.000391,...,0.000367,0.000264,0.000397,0.000280,0.001310,0.000519,0.000263,0.000395,0.000309,0.000572


In [17]:
''' Calculating the downside statistics.
Parameters
----------
:down_stat:  dataframe
    Contains the downside statistics.
-------
'''
#down_stat = pd.DataFrame(columns=stock_data['Symbol'], 
#                        index=['Semi Deviation', 'Gain Deviation', 'Loss Deviation', 'Downside Deviation (MAR=2.0%)',
#                               'Downside Deviation (rf=0.5%)', 'Downside Deviation (0%)', 'Maximum Drawdown', 
#                               'Historical VaR (95%)', 'Historical ES (95%)', 'Modified VaR (95%)', 'Modified ES (95%)'])

#for s in stock_data['Symbol']:
#    df = pd.DataFrame(returns_daily[s])
#    down_stat[s] = pat.create_downside_table(df,0)
#down_stat

' Calculating the downside statistics\nParameters\n----------\n:down_stat:  dataframe\n    Contains the downside statistics.\n-------\n'

___
## 2. Price Pressure <a class="anchor" id="5-bullet"></a>

__To-Do's__
* Welche Index Inklusion Effekte gibt es.
* Gibt es einen Pre-announcement drift? 
* Gibt es sonstige Announcement Effekte?

__Paper 1 EMH vs. PPH nachbauen__
* Alle Aktien für die Analyse zusammen suchen - bisher sind nur die neusten 40 enthalten (Sophie hat hier schon eine CSV vorbereitet).
*    Analyse excess return & trading volume on the first 5 days with the cross sectional means.
*    Analysis before and after annoucment as well as after the inclusion day

* Vorschläge hierzu von Stefan:
    * Alles Herausnahmen und Hereinnahmen in den DAX zusammennehmen, da nur so statistische Test möglich sind
    * Bspw. vor 2010, nach 2010 Veränderungen anschauen





Änderungsvorschläge von Stefan
	•	Alles Änderungen zusammen (30 auf 40)
--> inferenzen, statistischen tests nur so möglich
	•	Vor 2010, nach 2010
	•	Überlegen, wie wir das empirisch machen --> counterfactual
	•	Abnormal returns, worauf basiert es; was ist expected return; komplexeres modell
	•	Index Inklusion Effekte
	•	Freiheitsgrade im empirischen Ansatz, solange die Frage beantwortet wird wie sich Inklusion auf Expected Returns und andere Metriken auswirkt
	•	Abnormal returns; Volatilität; Handelsvolumen (ETFs müssen sie nun auch handeln); Investors attention (das sollten wir absprechen; ggf. Aufnahme in Index als Maß für Attention; dazu: googeln)
	•	Korrelationen: Wie ändern sich Korrelation (das könnte attention sein) --> ökonomisch bedeutsam, weil systematisches Risiko
	•	Ggf. Datenfiltern; Daten Fehler; für Preise funktioniert gut; Handelsvolumen nicht so zuverlässig für damals
	•	Wichtig, dass Tage stimmen, ansonsten problematisch
	•	Announcement day & inclusion day
	•	Pre-announcement drift
	•	Announcement Effekte
	•	Dividends announcements --> Literatur

___
## 3. Correlation Analysis <a class="anchor" id="6-bullet"></a>

In this Part, we'll have a closer look onto the correlation of a stock before and after inclusion/exclusion with the index itself.

In [149]:
data = returns_daily_included
data.update(returns_daily_excluded)
dax = returns_daily['^GDAXI']
unique_stocks = list(dict.fromkeys(list(info_df.loc[~info_df['Type'].isnull()]['Symbol'])))

corr_df_list = []
for s in unique_stocks:
    for t in info_df.loc[~info_df['Type'].isnull()].loc[info_df['Symbol'] == s]['Type']:
        temp_date = info_df.loc[info_df['Symbol'] == s]['Date'].values[0]
        temp_data_before = data[s].to_frame().loc[temp_date - np.timedelta64(365,'D'):temp_date]['Adj Close']
        temp_data_after = data[s].to_frame().loc[temp_date:temp_date + np.timedelta64(365,'D')]['Adj Close']
        temp_dax_before = dax.loc[temp_date - np.timedelta64(365,'D'):temp_date]
        temp_dax_after = dax.loc[temp_date:temp_date + np.timedelta64(365,'D')]
        corr_before = temp_data_before.corr(temp_dax_before)
        corr_after = temp_data_after.corr(temp_dax_after)
        if temp_date == np.datetime64('2021-09-20'):
            big_inc = True
        else:
            big_inc = False
        corr_df_list.append({'Ticker': s, 'Type': t, '30 -> 40': big_inc, 'Corr_before': corr_before, 'Corr_after': corr_after, 'Delta': corr_after - corr_before})

corr_df = pd.DataFrame(corr_df_list)
corr_df.sort_values(by=['Delta'], inplace=True)
corr_df

,Ticker,Type,30 -> 40,Corr_before,Corr_after,Delta
12,BEI.DE,Included,False,0.604934,0.318893,-0.286041
11,BEI.DE,Excluded,False,0.604934,0.318893,-0.286041
8,LHA.DE,Excluded,False,0.754511,0.487113,-0.267398
16,CON.DE,Included,False,0.846636,0.657267,-0.189369
20,DHER.DE,Included,False,0.359501,0.174265,-0.185237
4,PSM.DE,Excluded,False,0.403081,0.314127,-0.088954
5,PSM.DE,Included,False,0.403081,0.314127,-0.088954
0,SZG.DE,Excluded,False,0.752605,0.671941,-0.080664
10,WDI.HM,Included,False,0.138840,0.071607,-0.067234
9,WDI.HM,Excluded,False,0.138840,0.071607,-0.067234


In [145]:
len(list(dict.fromkeys(list(info_df.loc[~info_df['Type'].isnull()]['Symbol']))))
#s
#list(dict.fromkeys(list(info_df.loc[~info_df['Type'].isnull()]['Symbol'])))
len(corr_df)

27

___
## 4. Systematic Risk and Liquidity <a class="anchor" id="7-bullet"></a>

It is examined whether the inclusion of a share in the DAX affects the systematic risk and the liquidity of the share in question.

## Systematic Risk of all newly Stocks included in the DAX <a class="anchor" id="8-bullet"></a>

In [11]:
''' Calculating the systmatic risk after the inclusion of the Stocks in the DAX.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 2009-06-21
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''
i = 0
j = 1
sys_risk = []
while i in range(0,10):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_included.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_included[DAX_included.iloc[i][1]][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))]
    data['Dummy'] = benchmark['Dummy'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))]
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_included.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
  
while i in range(10,len(returns_daily_included)):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_included.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_included[DAX_included.iloc[i][1]][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data['Dummy'] = benchmark['Dummy'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_included.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()
        }, ignore_index=True
    )

,Stock,Rank,$\Delta$,p_Value,$R^{2}$
0,HEI.DE,1.0,0.103273,0.496684,0.379531
1,CON.DE,2.0,-0.321822,0.002120,0.634464
2,LXS.DE,3.0,-0.454091,0.000233,0.506027
3,VNA.DE,4.0,0.118351,0.302192,0.302195
4,PSM.DE,5.0,0.055706,0.498706,0.490919
5,1COV.DE,6.0,0.543596,0.002148,0.390857
6,WDI.HM,7.0,0.519190,0.095540,0.177506
7,MTX.DE,8.0,0.089401,0.509244,0.337730
8,DWNI.DE,9.0,-0.008597,0.956859,0.124476
9,DHER.DE,10.0,-0.104012,0.639293,0.085454


## Systematic Risk of all newly Stocks excluded in the DAX <a class="anchor" id="9-bullet"></a>

In [12]:
''' Calculating the systmatic risk after the exclusion of the Stocks in the DAX.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 2009-06-21
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''

i = 0
j = 1
sys_risk = []
while i in range(0,7):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_excluded.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_excluded[DAX_excluded.iloc[i][1]][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))]
    data['Dummy'] = benchmark['Dummy'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))]   
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_excluded.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
    
while i in range(7,len(returns_daily_excluded)):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_excluded.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_excluded[DAX_excluded.iloc[i][1]][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data['Dummy'] = benchmark['Dummy'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_excluded.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()            
        }, ignore_index=True
    )

,Stock,Rank,$\Delta$,p_Value,$R^{2}$
0,SZG.DE,1.0,0.009832,0.919350,0.516948
1,LXS.DE,2.0,0.130526,0.114502,0.565313
2,SDF.DE,3.0,0.209893,0.231217,0.233359
3,PSM.DE,4.0,-0.318287,0.178126,0.128139
4,CBK.DE,5.0,0.566617,0.000897,0.303567
5,TKA.DE,6.0,0.450349,0.015504,0.282978
6,LHA.DE,7.0,0.090756,0.728232,0.403611
7,WDI.HM,8.0,-0.093844,0.865141,0.015965
8,BEI.DE,9.0,-0.218337,0.014602,0.283706
9,DWNI.DE,10.0,0.460957,0.027038,0.059404


## Systematic Risk of the 10 Stocks from DAX Increase in 2021 <a class="anchor" id="10-bullet"></a>

In [13]:
''' Creating a list and dictionary with all 10 newly added DAX stocks.
Parameters
----------
:newcomers:  list
    Contains the names of the stocks.
:dax_new: dict
    Contains the daily returns of the 10 new stocks.
-------
'''
newcomers = ['AIR.DE', 'SHL.DE', 'ZAL.DE', 'SY1.DE', 'SRT3.DE',  'POAHY', 'HFG.DE', 'BNR.DE', 'QIA.DE', 'PUM.DE']
dax_new = {new: returns_daily[new] for new in newcomers}

In [14]:
''' Creating the dummy variable - 0 before the inclusion day (2021-09-20) and 1 thereafter.
Parameters
----------
:benchmark:  df
    Contains daily returns as well as the dummy variable.
-------
'''
d = []
for date in benchmark.index:
    if str(date) < '2021-09-20 00:00:00':
        d.append(0)
    else: d.append(1)
benchmark['Dummy'] = d

In [15]:
''' Calculating the systmatic risk.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 1 Year before the inclusion day (2020-09-20)
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''
i = 1
sys_risk = []
for key in dax_new:
    data = pd.DataFrame(dax_new[key]['2020-09-20':'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs']['2020-09-20':'2022-03-01']
    data['Dummy'] = benchmark['Dummy']['2020-09-20':'2022-03-01']
    stocks_as_df['Volume'][key]['2020-09-20':'2021-09-20']    
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': key,
            'Rank': i, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()
        }, ignore_index=True
    )

,Stock,Rank,$\Delta$,p_Value,$R^{2}$
0,AIR.DE,1.0,-0.115361,0.687517,0.428007
1,SHL.DE,2.0,0.139242,0.287608,0.133441
2,ZAL.DE,3.0,0.187244,0.580780,0.065300
3,SY1.DE,4.0,0.296144,0.061525,0.078697
4,SRT3.DE,5.0,0.498162,0.104634,0.033886
5,POAHY,6.0,0.042090,0.842130,0.290775
6,HFG.DE,7.0,0.886685,0.047033,0.035359
7,BNR.DE,8.0,0.203588,0.216613,0.352506
8,QIA.DE,9.0,0.222901,0.348398,0.006456
9,PUM.DE,10.0,0.069367,0.610297,0.274040


In [16]:
'''Distribution of the shares with a higher unit share in the DAX and all those with a weighting of < 1 %. 
Parameters
----------
:des_stat:  df
    N: Number of stocks sorted after the index weight.
    Mean: Mean systematic risk. 
    R^2: Mean R-squared of the model.
-------
'''
des_stat = []
des_stat.append(
        {
            'N': '1-5',
            r"$\varnothing$": sys_risk[:5][r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[:5][r"$R^{2}$"].mean()
        }
    )
des_stat.append(
        {
            'N': '6-10',
            r"$\varnothing$": sys_risk[5:][r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[5:][r"$R^{2}$"].mean()
        }
    )
pd.DataFrame(des_stat)

,N,$\varnothing$,$R^{2}$
0,1-5,0.201086,0.147866
1,6-10,0.284926,0.191827
